# Detecting Regional Seismic  Phases  

### Creating the input from the SAC files
This step can be for three components record or one component record

In [ ]:
import glob

sac_dir = '/localdata/rayan/dataprc_sac_hvsr_data/2012*/*.BHZ.D.SAC'
out_sac = open('sacfiles.txt', 'w')
for sacfile in (glob.glob(sac_dir)):
    if len(glob.glob(sacfile[:-9]+'*.SAC')) == 3:
        #out_sac.write('\n'.join([sacfile[:-9]+"BHZ.D.SAC  " +
        #                    sacfile[:-9]+"BHN.D.SAC  "+sacfile[:-9]+"BHE.D.SAC  "+'\n']))
        out_sac.write('\n'.join([sacfile[:-9]+"*BHZ.D.SAC"+'\n']))
    else:
        print ('there are no BHE BHN BHZ')
out_sac.close()

In [2]:
!pwd

/localdata/rayan/generalized-phase-detection


## Testing the input of the training data

In [ ]:
import string
import time
import argparse as ap
import sys
import os
import obsplus as op
import numpy as np
import obspy.core as oc
import matplotlib as mpl
import pylab as plt
import h5py
import time
from scipy.signal import spectrogram 
#import tensorflow as tf
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
##
freq_min=1
freq_max=10
filter_data = True
decimate_data = True

#
# Number of GPUs to use (if any)
n_gpu = 1 
#####################


half_dur = 0.64 
only_dt = 0.01
n_win = int(half_dur/only_dt)
n_feat = 2*n_win
n_feat_spec=65
n_shift = n_feat  # Number of samples to shift the sliding window at a time
sample_rate=1/only_dt
##using gpu here is much faster
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID" 
os.environ['CUDA_VISIBLE_DEVICES']="0" ##0=gpu,
config = tf.ConfigProto()
config.gpu_options.allow_growth=True
sess = tf.Session(config=config)
#using obsplus makes it faster

In [ ]:
#functions
def sliding_window(data, size, stepsize=128, padded=False, axis=-1, copy=False):
    #stepsize =400 so devided with 400 sample not over laping. 
    shape = list(data[0].shape)
    shape[axis] = np.floor(data[0][0].shape[axis] / stepsize - size / stepsize + 1).astype(int)
    shape.append(size)

    strides = list(data[0].strides)
    strides[axis] *= stepsize
    strides.append(data.strides[axis])
    print(("shape:{0},strides:{1}").format(shape,strides))

    strided = np.lib.stride_tricks.as_strided(data[0], shape=shape, strides=strides)
    
    if copy:
        return strided.copy()
    else:
        return strided
    
def spect(x):
    frequencies, times, spectrog = spectrogram(np.hstack(x),
                                              sample_rate,nfft=128,noverlap=0, 
                                              nperseg=128,window='triang',
                                              detrend='linear',
                                              scaling='spectrum'
                                             
                                             )
    return spectrog

#
def phases_to_train(st,freq_min,freq_max,op):
    st.detrend(type='linear')
   
    st.filter(type='bandpass', freqmin=freq_min, freqmax=freq_max)
    print(" filtering is done###")
   
    st.interpolate(100.0)
    print(" decimateing is done###")

    st_dict = {f'event{x}': st.copy() for x in range(1)}
    from_dict = op.obspy_to_array(st_dict)
    chan = st[0].stats.channel
    seedid=from_dict.coords['seed_id'].values[0]
    sr = from_dict.stats['event0'][seedid].sampling_rate
    dt = from_dict.stats['event0'][seedid].delta
    net = from_dict.stats['event0'][seedid].network
    sta = from_dict.stats['event0'][seedid].station

    t_p=from_dict.stats['event0'][seedid].sac.t1
    t_s1=from_dict.stats['event0'][seedid].sac.t5
    t_s2=from_dict.stats['event0'][seedid].sac.t4
    t_lg_b=from_dict.stats['event0'][seedid].sac.t2
    t_lg_e=from_dict.stats['event0'][seedid].sac.t3

    Noise_data=from_dict.sel(time=slice(0, t_p)).data

    P_data=from_dict.sel(time=slice(t_p, t_s1-2)).data

    S_data=from_dict.sel(time=slice(t_s2, t_lg_b)).data

    Lg_data=from_dict.sel(time=slice(t_lg_b, t_lg_e)).data
    
   
    Noise_train1=sliding_window(Noise_data, 128, stepsize=128)
    shp=(Noise_train1.shape[0],Noise_train1.shape[1],65)
    N_T=[spect(Noise_train1[:,i,:]) for i in range(0,Noise_train1.shape[1])]
    Noise_train=np.reshape(N_T, shp)
    #
    P_train1=sliding_window(P_data, 128, stepsize=128)
    shp=(P_train1.shape[0],P_train1.shape[1],65)
    P_T=[spect(P_train1[:,i,:]) for i in range(0,P_train1.shape[1])]
    P_train=np.reshape(P_T, shp)
    #
    S_train1=sliding_window(S_data, 128, stepsize=128)
    shp=(S_train1.shape[0],S_train1.shape[1],65)
    S_T=[spect(S_train1[:,i,:]) for i in range(0,S_train1.shape[1])]
    S_train=np.reshape(S_T, shp)
    #
    LG_train1=sliding_window(Lg_data, 128, stepsize=128)
    shp=(LG_train1.shape[0],LG_train1.shape[1],65)
    L_T=[spect(LG_train1[:,i,:]) for i in range(0,LG_train1.shape[1])]
    LG_train=np.reshape(L_T, shp)
    #
    tr_win0 = np.zeros((Noise_train.shape[1], n_feat_spec, 1))
    tr_type0 = np.zeros((Noise_train.shape[1]))
    tr_type0[:] = 0
    tr_win0[:, :, 0] = Noise_train[0]
    #tr_win0[:, :, 1] = Noise_train[1]
    #tr_win0[:, :, 2] = Noise_train[2]
    tr_win0 =tr_win0/np.max(np.abs(tr_win0),axis=(1,2))[:,None,None] #normalizine 
    
#
    tr_win1 = np.zeros((P_train.shape[1], n_feat_spec, 1))
    tr_type1 = np.zeros((P_train.shape[1]))
    tr_type1[:] = 1
    tr_win1[:, :, 0] = P_train[0]
    #tr_win1[:, :, 1] = P_train[1]
    #tr_win1[:, :, 2] = P_train[2]
    tr_win1 =tr_win1/np.max(np.abs(tr_win1),axis=(1,2))[:,None,None]#normalizine 
    #
    tr_win2 = np.zeros((S_train.shape[1], n_feat_spec, 1))
    tr_type2 = np.zeros((S_train.shape[1]))
    tr_type2[:] = 2
    tr_win2[:, :, 0] = S_train[0]
    #tr_win2[:, :, 1] = S_train[1]
    #tr_win2[:, :, 2] = S_train[2]
    tr_win2 =tr_win2/np.max(np.abs(tr_win2),axis=(1,2))[:,None,None]#normalizine                         
                            
                            
#
    tr_win3= np.zeros((LG_train.shape[1], n_feat_spec, 1))
    tr_type3 = np.zeros((LG_train.shape[1]))
    tr_type3[:] = 3
    tr_win3[:, :, 0] = LG_train[0]
    #tr_win3[:, :, 1] = LG_train[1]
    #tr_win3[:, :, 2] = LG_train[2]
    tr_win3 =tr_win3/np.max(np.abs(tr_win3),axis=(1,2))[:,None,None]#normalizine                         
   
    return Noise_train1,Noise_train ,tr_win0, tr_type0,tr_win1, tr_type1,tr_win2, tr_type2,tr_win3, tr_type3

In [ ]:
if __name__ == "__main__":    
    
    tr_win=np.zeros([1,n_feat_spec,1])
    tr_type=np.zeros([1])
    time0=time.time()
   
    with open('sacfiles.txt') as f:
 
       
        for count,line in enumerate (f.read().splitlines()):

            time1=time.time()
            ff=oc.read(str(line))
            Noise_train1,Noise_train,tr_win0, tr_type0,tr_win1, tr_type1,tr_win2, tr_type2,tr_win3, tr_type3=phases_to_train(ff,freq_min,freq_max,op)

            #np.fromiter(tr_type4,dtype="float")
            tr_win=np.concatenate((tr_win,tr_win0,tr_win1,tr_win2,tr_win3))
            tr_type=np.concatenate((tr_type,tr_type0,tr_type1,tr_type2,tr_type3))
            time2=time.time()
            total_time=time2-time1
            print('time for line number {} is {} second'.format(count,total_time))
          
    
time3=time.time()
time=time3-time0
print('all_time is {} second'.format(time))

In [ ]:
with h5py.File('data100fs_0.5_10HZ_spec.h5', 'w') as  hf:
    hf.create_dataset('X', data=tr_win)
    hf.create_dataset('Y', data=tr_type)
#read the hf5 data to check the output

"""
with h5py.File('data.h5', 'r') as of:
    X=of.get('X')
    Y=of.get('Y')
"""

In [ ]:

with h5py.File('data100fs_0.5_10HZ_spec.h5', 'r') as of:
    X=of.get('X')
    Y=of.get('Y')
    print (of['Y'])
    print (of['X'])
    print (of['X'][700].shape)
    #print (np.unique(of['X'][1:]))
    sample_count1=of['X'].shape[0]
    print(sample_count1)

## Training model

In [ ]:
#based on [Ross et al., 2018]
import numpy as np
import pylab as plt
import warnings
import os
import matplotlib as mpl
# mpl.use('Agg')
import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten
from tensorflow.keras.layers import Conv1D, MaxPooling1D, LSTM
from tensorflow.keras import losses
from tensorflow.keras.models import model_from_json
from keras.utils.np_utils import to_categorical
#import tensorflow as tf
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

# important steps
config = tf.ConfigProto()
config.gpu_options.allow_growth=True
sess = tf.Session(config=config)
#
mpl.rcParams['pdf.fonttype'] = 42

warnings.simplefilter("ignore")

#hdf_file = "scsn_ps_2000_2017_shuf.hdf5"
hdf_file = "data100fs_0.5_10HZ_spec.h5"
sample_count = sample_count1 #2421000 ##total number of 400 windows we have. 
show_plot = 1
train = 1
total_dur = 1.28
win_dur = 1.28  
dt = 0.01
phase_dir = "phase"
#n_win = int(win_dur / dt)
n_win=int(65)
#n_full = int(total_dur / dt)
n_full=int(65)
n_start = (n_full - n_win) // 2
gpu_count=1
batchsize=1000
epoches=15

In [ ]:
#functions 
def make_parallel(model, gpu_count):
    """"
    This should be only used when we have many GPU
    """
    from tensorflow.keras.layers import add
    from tensorflow.keras.layers import Lambda
    from tensorflow.keras.models import Model

    def get_slice(data, idx, parts):
        shape = tf.shape(data)
        size = tf.concat([shape[:1] // parts, shape[1:]], axis=0)
        stride = tf.concat([shape[:1] // parts, shape[1:]*0], axis=0)
        start = stride * idx
        return tf.slice(data, start, size)

    outputs_all = []
    for i in range(len(model.outputs)):
        outputs_all.append([])

    # Place a copy of the model on each GPU, each getting a slice of the batch
    for i in range(gpu_count):
        with tf.device('/gpu:%d' % i):
            with tf.name_scope('tower_%d' % i) as scope:

                inputs = []
                # Slice each input into a piece for processing on this GPU
                for x in model.inputs:
                    input_shape = tuple(x.get_shape().as_list())[1:]
                    slice_n = Lambda(get_slice, output_shape=input_shape, arguments={
                                     'idx': i, 'parts': gpu_count})(x)
                    inputs.append(slice_n)

                outputs = model(inputs)

                if not isinstance(outputs, list):
                    outputs = [outputs]

                # Save all the outputs for merging back together later
                for l in range(len(outputs)):
                    outputs_all[l].append(outputs[l])
                print('ff')
                print(outputs_all)

    # merge outputs on CPU
    with tf.device('/cpu:0'):
        merged = []
        for outputs in outputs_all:
            #merged.append(merge(outputs, mode='concat', concat_axis=0))
            merged.append(tf.keras.layers.Concatenate(axis=0)(outputs))
           

        return Model(model.inputs, merged)


def train_model(model, seis_data_fn="scsn_all.hdf5",
                validation_ratio=0.25, batch_size=32,
                sample_count=None, callbacks=None,epochs=100):
    """ Train the classification model:

    """
    with h5py.File(seis_data_fn, "r") as seis_data:
        if sample_count is None:
            sample_count = int(seis_data.attrs["sample_count"])
        sample_idxs = range(0, sample_count)
        #sample_idxs = np.random.permutation(sample_idxs)
        train_samp_idxs = sample_idxs[0:int(
            (1 - validation_ratio) * sample_count)]
        val_samp_idxs = sample_idxs[int(
            (1 - validation_ratio) * sample_count):]
        print("Using %d samples for training set" % len(train_samp_idxs))
        print("Using %d samples for validation set" % len(val_samp_idxs))
        train_seq_gen = gen_train_seq(batch_size=batch_size,
                                      seis_data=seis_data,
                                      train_samp_idxs=train_samp_idxs)
        val_seq_gen = gen_val_seq(batch_size=batch_size,
                                  seis_data=seis_data,
                                  val_samp_idxs=val_samp_idxs)
        hist = model.fit_generator(generator=train_seq_gen,
                                   validation_data=val_seq_gen,
                                   steps_per_epoch=len(
                                       train_samp_idxs)/batch_size,
                                   validation_steps=len(
                                       val_samp_idxs)/batch_size,
                                   epochs=epochs,
                                   max_queue_size=1000,
                                   verbose=1,
                                   workers=1,
                                   use_multiprocessing=False,
                                   callbacks=callbacks)
        return hist


def predict_test(model, seis_data_fn="scsn_all.hdf5",
                 validation_ratio=0.3, batch_size=32,
                 sample_count=None):
    """ predict the classification model
    """
    with h5py.File(seis_data_fn, "r") as seis_data:
        if sample_count is None:
            sample_count = int(seis_data.attrs["sample_count"])
        sample_idxs = range(0, sample_count)
        #sample_idxs = np.random.permutation(sample_idxs)
        val_samp_idxs = sample_idxs[int(
            (1 - validation_ratio) * sample_count):]
        print("Using %d samples for validation set" % len(val_samp_idxs))
        val_seq_gen = gen_val_seq(batch_size=batch_size,
                                  seis_data=seis_data,
                                  val_samp_idxs=val_samp_idxs)
        y_pred = model.predict_generator(generator=val_seq_gen,
                                         steps=len(val_samp_idxs)/batch_size,
                                         max_queue_size=1000,
                                         verbose=1,
                                         workers=1,
                                         use_multiprocessing=False)
        val_seq_gen_y = gen_val_seq_y(batch_size=batch_size,
                                      seis_data=seis_data,
                                      val_samp_idxs=val_samp_idxs)
        Y_test = []
        snr = []
        dist = []
        mag = []
        count = 0
        for vals in val_seq_gen_y:
            count += 1
            Y_test += vals.tolist()
            #Y_test += vals[0].tolist()
            #snr += vals[1].tolist()
            #dist += vals[2].tolist()
            #mag += vals[3].tolist()
            if count >= len(val_samp_idxs)/batch_size:
                break

    # , np.array(snr), np.array(dist), np.array(mag)
    return np.array(y_pred), np.array(Y_test).reshape(len(Y_test), 1)


def gen_train_seq(batch_size, seis_data, train_samp_idxs):
    """ Generates training data 
    """
    while True:
        # generate sequences for training
        train_samp_count = len(train_samp_idxs)
        batches = int(train_samp_count / batch_size)
        remainder_samples = train_samp_count % batch_size
        if remainder_samples:
            batches = batches + 1
        # generate batches of samples
        for idx in range(0, batches):
            if idx == batches - 1:
                batch_idxs = train_samp_idxs[idx * batch_size:]
            else:
                batch_idxs = train_samp_idxs[idx * batch_size:idx
                                             * batch_size + batch_size]
            batch_idxs = sorted(batch_idxs)

            X = seis_data["X"][batch_idxs, n_start:n_start+n_win]
            Y = seis_data["Y"][batch_idxs]
            yield (np.array(X).reshape((X.shape[0], X.shape[1], 1)),
                   to_categorical(Y, 4))#number of the classes


def gen_val_seq(batch_size, seis_data, val_samp_idxs):
    """ Generates validation data
    """
    while True:
        # generate sequences for validation
        val_samp_count = len(val_samp_idxs)
        batches = int(val_samp_count / batch_size)
        remainder_samples = val_samp_count % batch_size
        if remainder_samples:
            batches = batches + 1
        # generate batches of samples
        for idx in range(0, batches):
            if idx == batches - 1:
                batch_idxs = val_samp_idxs[idx * batch_size:]
            else:
                batch_idxs = val_samp_idxs[idx * batch_size:idx
                                           * batch_size + batch_size]
            batch_idxs = sorted(batch_idxs)

            X = seis_data["X"][batch_idxs, n_start:n_start+n_win]
            Y = seis_data["Y"][batch_idxs]
            yield (np.array(X).reshape((X.shape[0], X.shape[1], 1)),
                   to_categorical(Y, 4))#classes


def gen_val_seq_y(batch_size, seis_data, val_samp_idxs):
    """ Generates validation data
    """
    while True:
        # generate sequences for validation
        val_samp_count = len(val_samp_idxs)
        batches = int(val_samp_count / batch_size)
        remainder_samples = val_samp_count % batch_size
        if remainder_samples:
            batches = batches + 1
        # generate batches of samples
        for idx in range(0, batches):
            if idx == batches - 1:
                batch_idxs = val_samp_idxs[idx * batch_size:]
            else:
                batch_idxs = val_samp_idxs[idx * batch_size:idx
                                           * batch_size + batch_size]
            batch_idxs = sorted(batch_idxs)

            Y = seis_data["Y"][batch_idxs]

            yield np.array(Y) 


def WritePhaseFile(fdir, evid, fm):
    """ Writes out an STP format phase file using picked first motions"""
    line_count = 0
    picks = {}
    g = open("%s/%s.phase" % (fdir, evid), 'w')
    with open("%s/%s.phase" % (phase_dir, evid), 'r') as f:
        for line in f:
            if line_count > 0:
                temp = line.split()
                net = temp[0]
                sta = temp[1]
                chan = temp[2]
                if chan[0] == 'B':
                    chan = list(chan)
                    chan[0] = 'H'
                    chan = "".join(chan)
                phase = temp[7]
                pick = temp[12]
                pol = temp[8][0]
                temp2 = line
                if phase == 'P':
                    if chan[2] != 'Z':
                        chan = list(chan)
                        chan[2] = 'Z'
                        chan = "".join(chan)
                    temp2 = list(temp2)
                    if (net, sta, chan) in fm:
                        temp2[49] = fm[(net, sta, chan)]
                    else:
                        temp2[49] = '.'
                    temp2 = "".join(temp2)
                g.write(temp2)
            else:
                g.write(line)
            line_count += 1
    g.close()
    return picks

In [ ]:

if __name__ == "__main__":
    import tensorflow.compat.v1 as tf
    tf.disable_v2_behavior()

    from sklearn.externals import joblib
    import h5py
    from tensorflow.compat.v1.keras.layers import BatchNormalization
    from tensorflow.compat.v1.keras.layers import MaxPooling1D
    from tensorflow.compat.v1.keras.layers import GaussianNoise
   
    if train:
        model = Sequential()
        model.add(Conv1D(32, 21, padding='same', input_shape=(n_win, 1)))
        model.add(BatchNormalization())
        model.add(Activation('relu'))
        model.add(MaxPooling1D(pool_size=2))

        model.add(Conv1D(64, 15, padding='same'))
        model.add(BatchNormalization())
        model.add(Activation('relu'))
        model.add(MaxPooling1D(pool_size=2))

        model.add(Conv1D(128, 11, padding='same'))
        model.add(BatchNormalization())
        model.add(Activation('relu'))
        model.add(MaxPooling1D(pool_size=2))

        model.add(Conv1D(256, 9, padding='same'))
        model.add(BatchNormalization())
        model.add(Activation('relu'))
        model.add(MaxPooling1D(pool_size=2))
   
        model.add(Flatten())

        model.add(Dense(200))
        model.add(BatchNormalization())
        model.add(Activation('relu'))

        model.add(Dense(200))
        model.add(BatchNormalization())
        model.add(Activation('relu'))

        model.add(Dense(4)) #4 is number the output samples N pn sn Lg
        model.add(Activation('softmax')) # software for multiple sampels 

        print(model.summary())
        adam = tf.keras.optimizers.Adam()

        #model = make_parallel(model, gpu_count)#in case we have many GPUS

        model.compile(loss=losses.categorical_crossentropy,
                      optimizer=adam, metrics=['accuracy'])

        early_stop = tf.keras.callbacks.EarlyStopping(patience=5,
                                                   monitor='val_acc') #monitoring the accuracy
        model_cp = tf.keras.callbacks.ModelCheckpoint("model_pol_best.hdf5",
                                                   # model_cp = keras.callbacks.ModelCheckpoint("model.{epoch:02d}-{val_loss:.2f}.hdf5",
                                                   save_best_only=True)
       
        hist = train_model(model, seis_data_fn=hdf_file,
                           validation_ratio=0.25, batch_size=batchsize,
                           callbacks=[early_stop, model_cp],
                           sample_count=sample_count,epochs=epoches)

        hist = hist.history
        joblib.dump(hist, "hist_pol.pkl")

        print("Saved picking model to disk")

        # serialize model to JSON
        model_json = model.to_json()
        with open("model_pol.json", "w") as json_file:
            json_file.write(model_json)

        # serialize weights to HDF5
        # model.save_weights("model_pick_best.hdf5")

## Prediction code 


In [ ]:
    # load json and create model
    json_file = open('model_pol.json', 'r')
    loaded_model_json = json_file.read()
    json_file.close()
    model = model_from_json(loaded_model_json, custom_objects={
        'tf': tf})
    # load weights into new model
    model.load_weights("model_pol_best.hdf5")
    #model = make_parallel(model, 1)
    hist = joblib.load("hist_pol.pkl")
    print("Loaded model from disk")

    if True:
        plt.clf()
        fig, ax = plt.subplots(1, 1)
        ax.plot(np.arange(len(hist['loss'])), hist['loss'], c='k', marker='o',
                label='Training')
        ax.plot(np.arange(len(hist['val_loss'])), hist['val_loss'], c='b',
                marker='o', label='Validation')
        ax.set_xlabel("Epoch")
        ax.set_ylabel("Loss")
        ax.legend(loc=0)
        """"
        if show_plot:
            plt.show()
        else:
            plt.savefig("pick_loss.pdf")
         """
        plt.savefig("pick_loss.pdf")
        plt.show()

    print("Running forward predictions on test data")
    y_pred_proba, y_test = predict_test(model, seis_data_fn=hdf_file,
                                        # y_pred, y_test, snr, dist, mag = predict_test(model, seis_data_fn=hdf_file,
                                        validation_ratio=0.8,
                                        batch_size=10000,
                                        sample_count=sample_count)
    y_pred = np.argmax(y_pred_proba, axis=1)
    print("")
    print("Finished predictions")

    from sklearn.metrics import classification_report
    target_names = ['Noise', 'P','S','Lg']
    print(classification_report(y_test, y_pred, labels=[0,1,2,3]))
    y_test = y_test.flatten()
    FP = []
    FN = []
    for pr_min in np.arange(0, 1.0, 0.01):
        y_pred0 = np.argmax(y_pred_proba, axis=1)
        y_prob0 = np.max(y_pred_proba, axis=1)
                       
            

        idx = np.where(y_prob0 < pr_min)[0]
        y_pred0[idx] = 2

        idx = np.where(y_pred0 < 2)[0]
        precision = np.sum(y_pred0[idx] == y_test[idx]) / float(idx.size)
        idx = np.where(y_test < 2)[0]
        recall = np.sum(y_pred0[idx] == y_test[idx]) / float(idx.size)
        FP.append(precision)
        FN.append(recall)
        

In [ ]:
    pr_min = np.arange(0, 1.0, 0.01)
    plt.scatter(FN, FP, c=pr_min, cmap='jet')
    plt.xlabel("Recall")
    plt.ylabel("Precision")
    plt.colorbar()
    
    
    plt.savefig("recall_precision.pdf")
    plt.xlim(0.1, 1.0)
    plt.ylim(0.1, 1.0)
    plt.show()